<a href="https://colab.research.google.com/github/GoAshim/Deep-Learning-with-TensorFlow/blob/main/03_Regression_Problem_with_TensorFlow_Predict_Premium.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Solving Regression Problem with Neural Networks using TensorFlow
In this project we will use neural networks in TensorFlow with a simple dataset to predict a numerical variable (or target feature) based on one or more other variables (input features). The scope of this project is to learn the fundamentals of neural network in tensorflow by going through
the steps to build and train the model and use the model to predict result with unknown data.

In [9]:
# Import libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf

# Check version of tensorflow
tf.__version__

'2.15.0'

### Loading data
In this project we will create a very simple dataset with one feature and one label.

In [3]:
# Create feature data
X = np.array([-7., -4., -1., 2., 5., 8., 11., 14.])

# Create label data
y = np.array([3., 6., 9., 12., 15., 18., 21., 24.])

X, y

(array([-7., -4., -1.,  2.,  5.,  8., 11., 14.]),
 array([ 3.,  6.,  9., 12., 15., 18., 21., 24.]))

In [4]:
# Convert both features and labels from numpy array to tensor
X = tf.constant(X)
y = tf.constant(y)
X, y

(<tf.Tensor: shape=(8,), dtype=float64, numpy=array([-7., -4., -1.,  2.,  5.,  8., 11., 14.])>,
 <tf.Tensor: shape=(8,), dtype=float64, numpy=array([ 3.,  6.,  9., 12., 15., 18., 21., 24.])>)

### Creating deep learning model with TensorFlow
We are going to run the following steps -
* **Creating the model** - We will create the model by defining the following of a neural network model.
  * Input layer
  * Output layer
  * Hidden layer(s) - One or more layers in between the input and output layer.
* **Compiling the model** - We will compile the model by defining the following of a neural network model.
  * Loss function - This tells our model how much wrong it is.
  * Optimizer - This tells our model how to reduce the loss, in other words how to improve it's performance.
  * Evaluation metrics - Helps to evaluate performance of the model.
* **Fitting the model** - We will run the model with data and let it find pattern between feature(s) and target. Here we will specify -
  * Epoch - The number of times the model will run on data to find the pattern.

In [8]:
# Set the random seed
tf.random.set_seed(42)

# Create a sequential model and one dense layer
model = tf.keras.Sequential()
model.add(tf.keras.layers.Dense(1))

# Compile the model
model.compile(loss = tf.keras.losses.mae, # mae = Mean Absolute Error
              optimizer = tf.keras.optimizers.SGD(),
              metrics = ["mae"])

# Fit

(<tf.Tensor: shape=(2, 3), dtype=float32, numpy=
 array([[1., 1., 1.],
        [1., 1., 1.]], dtype=float32)>,
 TensorShape([2, 3]),
 TensorShape([8]),
 2,
 1)